# Passing Data Between Tests

In this notebook we will demonstrate how to make use of the test context to pass data between tests. Any object can be passed to the context, but we will use a simple dictionary to demonstrate the concept.

In [ ]:
%load_ext dotenv
%dotenv dev.env

import validmind as vm

vm.init(
  api_host = "https://api.prod.validmind.ai/api/v1/tracking",
  # Use your project ID
  project = "..."
)


## Build Some Sample Metrics

We will build two simple metrics to demonstrate the concept. The first metric will create a sample dataframe with mock data, and the second will add one more column to it.

In [ ]:
import pandas as pd
from dataclasses import dataclass
from validmind.vm_models import Metric

@dataclass
class MyFirstMetric(Metric):
    type = "dataset"
    key = "my_first_metric"

    def run(self):
        df = pd.DataFrame({"column_a": [1, 2, 3, 4, 5]})
        # Store the dataframe in the test context
        self.test_context.set_context_data("some_dataset", df)

        return self.cache_results(df.to_dict("records"))

In [ ]:
from dataclasses import dataclass
from validmind.vm_models import Metric

@dataclass
class MySecondMetric(Metric):
    type = "dataset"
    key = "my_second_metric"

    def run(self):
        # Get the dataframe from the test context. We can
        # throw an error if it doesn't exist just to be sure
        df = self.test_context.get_context_data("some_dataset")
        if df is None:
            raise ValueError("'some_dataset' not found in test context")

        new_df = df.copy()
        new_df["column_b"] = [5, 4, 3, 2, 1]

        return self.cache_results(new_df.to_dict("records"))

Now let's define a test plan that will run the two metrics in sequence. We need to make sure that the first metric is run before the second one.

In [ ]:
from validmind.vm_models import TestPlan

class MyCustomTestPlan(TestPlan):
    """
    Custom test plan
    """

    name = "my_custom_test_suite"
    required_inputs = []
    tests = [MyFirstMetric, MySecondMetric]

my_custom_test_suite = MyCustomTestPlan()
my_custom_test_suite.run()